In [3]:
#Importamos las librerías
import ee
import geemap

In [4]:
#Nos autenticamos y iniciamos ee
ee.Authenticate()


Successfully saved authorization token.


In [5]:
ee.Initialize()

In [6]:
#Creamos nuestro primer mapa para visualizar
Map = geemap.Map()

In [7]:
# 1. Definir un punto (Vertedero de Burgos)

point = ee.Geometry.Point([-3.663863749008054
, 42.31458303530395])

# Creamos un área de 10x10 metros alrededor del punto
# Buffer de 5 m -> bounds() -> rectángulo de 10m x 10m
aoi_10m = point.buffer(5).bounds()

# 2. Cargar Sentinel-2 Harmonized
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate("2025-01-01", "2025-10-01")   # rango de fechas
      .filterBounds(point)
      .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)))  # menos del 5% de nubes

img_cant =  s2.count().getInfo()  # Número de imágenes
# Sacamos la media imagen
image = s2.median().clip(aoi_10m)

# 3. Calcular reflectancias medias en el área 10x10 m

vals = image.reduceRegion(
    reducer=ee.Reducer.mean(),   
    geometry=aoi_10m,
    scale=10
).getInfo()

print("Reflectancias medias en área 10x10 m:", vals)


Reflectancias medias en área 10x10 m: {'AOT': 182.5, 'B1': 1017.9795321637428, 'B11': 4085.5760233918127, 'B12': 2951.0058479532163, 'B2': 1620.3040935672516, 'B3': 1946.7894736842106, 'B4': 2615.0643274853796, 'B5': 3077.9853801169593, 'B6': 3199.472222222222, 'B7': 3412.980994152047, 'B8': 3452.5877192982457, 'B8A': 3751.3918128654973, 'B9': 3543.9269005847955, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 165.280701754386, 'TCI_G': 198.23684210526318, 'TCI_R': 249.65204678362574, 'WVP': 1343.7134502923977}


In [8]:
# Convertimos el punto a proyección UTM (EPSG:25830)
point_proj = point.transform('EPSG:25830', 1)

coords = point_proj.coordinates().getInfo()
x, y = coords  # en metros UTM

cell_size = 10  # metros

offsets = {
    "center": (0, 0),
    "N": (0, cell_size),
    "NE": (cell_size, cell_size),
    "E": (cell_size, 0),
    "SE": (cell_size, -cell_size),
    "S": (0, -cell_size),
    "SW": (-cell_size, -cell_size),
    "W": (-cell_size, 0),
    "NW": (-cell_size, cell_size)
}

results_dict = {}

for name, (dx, dy) in offsets.items():
    shifted = ee.Geometry.Point([x + dx, y + dy], 'EPSG:25830')
    aoi = shifted.buffer(5).bounds()

    vals = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10,
        crs='EPSG:25830',
        maxPixels=1e9
    ).getInfo()
    
    results_dict[name] = vals


In [9]:
print(results_dict)

{'center': {'AOT': 182.5, 'B1': 1019.7569169960475, 'B11': 4100.683794466404, 'B12': 2993.916996047431, 'B2': 1646.7628458498025, 'B3': 1993.604743083004, 'B4': 2695.086956521739, 'B5': 3100.9644268774705, 'B6': 3234.448616600791, 'B7': 3447.195652173913, 'B8': 3539.505928853755, 'B8A': 3782.4960474308305, 'B9': 3593.693675889328, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 168.08498023715413, 'TCI_G': 203.07509881422925, 'TCI_R': 251.99802371541503, 'WVP': 1346.887351778656}, 'N': {'AOT': 182.5, 'B1': 1028.5, 'B11': 4175, 'B12': 3205, 'B2': 1653.7578947368422, 'B3': 2047.105263157895, 'B4': 2827, 'B5': 3214, 'B6': 3406.5, 'B7': 3615.5, 'B8': 3697.9684210526316, 'B8A': 3935.5, 'B9': 3838.5, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 168.87894736

In [10]:
# 1. Definir un punto (Vertedero de Burgos)
long = -3.663863749008054
lat = 42.31458303530395

# Convertir 10 metros a grados aproximadamente
# 1 grado de latitud ≈ 111,000 metros
# 1 grado de longitud ≈ 111,000 * cos(latitud) metros
cell_size_lat = 10 / 111000  # ~0.00009 grados
cell_size_long = 10 / (111000 * abs(lat * 3.14159 / 180))  # ajustado por latitud


offsets = {
    "center": (long, lat),
    "N": (long, lat + cell_size_lat),
    "NE": (long + cell_size_long, lat + cell_size_lat),
    "E": (long + cell_size_long, lat),
    "SE": (long + cell_size_long, lat - cell_size_lat),
    "S": (long, lat - cell_size_lat),
    "SW": (long - cell_size_long, lat - cell_size_lat),
    "W": (long - cell_size_long, lat),
    "NW": (long - cell_size_long, lat + cell_size_lat)
}

point = ee.Geometry.Point(offsets["center"])

# 2. Cargar Sentinel-2 Harmonized
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterDate("2025-01-01", "2025-10-01")   # rango de fechas
      .filterBounds(point)
      .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 5)))  # menos del 5% de nubes

image = s2.first()
print("Imagen usada:", image.get("PRODUCT_ID").getInfo())

results_dict = {}

for name, coords in offsets.items():
    # Creamos un punto y area para cada offset
    point = ee.Geometry.Point(coords)
    aoi = point.buffer(5).bounds()

    # Extraemos las reflectancias medias
    vals = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=aoi,
        scale=10
    ).getInfo()
    
    results_dict[name] = vals
    print(f"Offset {name} ({coords[0]:.8f}, {coords[1]:.8f}): {vals}")

Imagen usada: S2B_MSIL2A_20250115T111319_N0511_R137_T30TVM_20250115T133241
Offset center (-3.66386375, 42.31458304): {'AOT': 57, 'B1': 472.1699604743083, 'B11': 2108.1581027667985, 'B12': 1450.810276679842, 'B2': 835.1541501976285, 'B3': 995.8102766798419, 'B4': 1462.086956521739, 'B5': 1582.0395256916995, 'B6': 1730.8893280632412, 'B7': 1899.723320158103, 'B8': 2118.2924901185775, 'B8A': 2214.490118577075, 'B9': 2080.474308300395, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0, 'MSK_CLASSI_SNOW_ICE': 0, 'MSK_CLDPRB': 0, 'MSK_SNWPRB': 0, 'QA10': None, 'QA20': None, 'QA60': 0, 'SCL': 5, 'TCI_B': 84.17391304347827, 'TCI_G': 102.82213438735177, 'TCI_R': 149.42687747035572, 'WVP': 337.3201581027668}
Offset N (-3.66386375, 42.31467313): {'AOT': 57, 'B1': 529, 'B11': 1992, 'B12': 1369, 'B2': 675.5138339920949, 'B3': 798.4901185770751, 'B4': 1190.505928853755, 'B5': 1316, 'B6': 1433, 'B7': 1550, 'B8': 1830.5533596837945, 'B8A': 1886, 'B9': 2206, 'MSK_CLASSI_CIRRUS': 0, 'MSK_CLASSI_OPAQUE': 0,

In [13]:
# Punto del vertedero Koshe (Addis Ababa)
koshe = ee.Geometry.Point([-3.663863749008054,42.31458303530395])

# Colección de embeddings AlphaEarth
embeddings = ee.ImageCollection('GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL')
year = 2024

# Crear un mosaico del año
mosaic = embeddings.filterDate(f'{year}-01-01', f'{year + 1}-01-01').mosaic()

# Obtener nombres de bandas
band_names = mosaic.bandNames()
print("Bandas disponibles:", band_names.getInfo())

# Extraer una muestra del punto
sample = mosaic.sample(region=koshe, scale=10, numPixels=1).first()

# Obtener los valores de la muestra como diccionario
sample_values = sample.toDictionary()

# Convertir la muestra a una imagen constante para cada banda
sample_img = ee.Image.constant(0)
band_list = band_names.getInfo()

for band in band_list:
    band_value = sample_values.get(band)
    band_img = ee.Image.constant(band_value).rename(band)
    sample_img = sample_img.addBands(band_img)

# Remover la banda constante inicial
sample_img = sample_img.select(band_list)

# Calcular la similitud (producto punto entre el píxel y el mosaico)
similarity = mosaic.multiply(sample_img).reduce(ee.Reducer.sum())

# Visualización
palette = ['000004', '2C105C', '711F81', 'B63679', 'EE605E', 'FDAE78', 'FCFDBF', 'FFFFFF']
vis_params = {'palette': palette, 'min': 0, 'max': 1}

# Mostrar en el mapa
Map.addLayer(similarity, vis_params, 'bright = more similar')
Map.centerObject(koshe, 11)
Map.addLayer(koshe, {'color': 'red'}, 'Koshe Landfill')
Map.setOptions('SATELLITE')
Map


Bandas disponibles: ['A00', 'A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A24', 'A25', 'A26', 'A27', 'A28', 'A29', 'A30', 'A31', 'A32', 'A33', 'A34', 'A35', 'A36', 'A37', 'A38', 'A39', 'A40', 'A41', 'A42', 'A43', 'A44', 'A45', 'A46', 'A47', 'A48', 'A49', 'A50', 'A51', 'A52', 'A53', 'A54', 'A55', 'A56', 'A57', 'A58', 'A59', 'A60', 'A61', 'A62', 'A63']


Map(bottom=388296.0, center=[42.31458303530395, -3.6638637490080543], controls=(WidgetControl(options=['positi…